In [2]:
#!conda list

In [3]:
import pygrib
import pandas as pd
import numpy as np

In [4]:
grbs = pygrib.open('20151018_1024_9b49a5d5.grib')

In [5]:
grb=grbs[1:]
grb.append(grbs[len(grb)+1])

In [6]:
#selected_grbs=grbs.select(shortName="u",typeOfLevel='isobaricInhPa',level=[500,700,800,925,1000])
#selected_grbs[0]

In [7]:
numChannels=12
gridShape=(221,381)
framesPerDay=24
isoLvls=[500,700,850,925,1000]
numIsoLvls=len(isoLvls)
numFrames=len(grb)
numDays=numFrames//(numChannels*framesPerDay*numIsoLvls)
#INDEX=Frame*5*12+IsoLvl*12+Channel (tot=12*5*24*numDays)
assert numDays==numFrames/(numChannels*framesPerDay*numIsoLvls) #Having integer number of days
assert grb[1].values.shape==gridShape #Check grid shape matches

In [8]:
firstFrame=grb[0]['day']

In [9]:
grb[0].keys()

['globalDomain',
 'GRIBEditionNumber',
 'eps',
 'offsetSection0',
 'section0Length',
 'totalLength',
 'editionNumber',
 'WMO',
 'productionStatusOfProcessedData',
 'section1Length',
 'wrongPadding',
 'table2Version',
 'centre',
 'centreDescription',
 'generatingProcessIdentifier',
 'gridDefinition',
 'indicatorOfParameter',
 'parameterName',
 'parameterUnits',
 'indicatorOfTypeOfLevel',
 'pressureUnits',
 'typeOfLevelECMF',
 'typeOfLevel',
 'level',
 'yearOfCentury',
 'month',
 'day',
 'hour',
 'minute',
 'second',
 'unitOfTimeRange',
 'P1',
 'P2',
 'timeRangeIndicator',
 'numberIncludedInAverage',
 'numberMissingFromAveragesOrAccumulations',
 'centuryOfReferenceTimeOfData',
 'subCentre',
 'paramIdECMF',
 'paramId',
 'cfNameECMF',
 'cfName',
 'cfVarNameECMF',
 'cfVarName',
 'unitsECMF',
 'units',
 'nameECMF',
 'name',
 'decimalScaleFactor',
 'setLocalDefinition',
 'optimizeScaleFactor',
 'dataDate',
 'year',
 'dataTime',
 'julianDay',
 'stepUnits',
 'stepType',
 'stepRange',
 'startSte

In [10]:
grb[0:13]

[1:Divergence:s**-1 (instant):regular_ll:isobaricInhPa:level 500:fcst time 0 hrs:from 201510180000,
 2:Fraction of cloud cover:(0 - 1) (instant):regular_ll:isobaricInhPa:level 500:fcst time 0 hrs:from 201510180000,
 3:Geopotential:m**2 s**-2 (instant):regular_ll:isobaricInhPa:level 500:fcst time 0 hrs:from 201510180000,
 4:Potential vorticity:K m**2 kg**-1 s**-1 (instant):regular_ll:isobaricInhPa:level 500:fcst time 0 hrs:from 201510180000,
 5:Specific cloud ice water content:kg kg**-1 (instant):regular_ll:isobaricInhPa:level 500:fcst time 0 hrs:from 201510180000,
 6:Specific cloud liquid water content:kg kg**-1 (instant):regular_ll:isobaricInhPa:level 500:fcst time 0 hrs:from 201510180000,
 7:Specific humidity:kg kg**-1 (instant):regular_ll:isobaricInhPa:level 500:fcst time 0 hrs:from 201510180000,
 8:Temperature:K (instant):regular_ll:isobaricInhPa:level 500:fcst time 0 hrs:from 201510180000,
 9:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 500:fcst time 0 hrs:

In [11]:
arr=np.zeros(shape=(numDays*framesPerDay,numIsoLvls,numChannels,*gridShape))
print(arr.shape)

def parseIndices(i):
    timeLoc=i//(numChannels*numIsoLvls)
    isoLoc=(i-timeLoc*(numChannels*numIsoLvls))//numChannels
    channelLoc=i-timeLoc*(numChannels*numIsoLvls)-isoLoc*numChannels
    return timeLoc,isoLoc,channelLoc

#INDEX=Frame*60+IsoLvl*12+Channel (tot=12*5*24*numDays)
for i,v in enumerate(grb):
    arr[*parseIndices(i)] = v.values

(168, 5, 12, 221, 381)


In [12]:
parseIndices(247)

(4, 0, 7)

In [13]:
grb[247].values

array([[242.90100098, 242.8873291 , 242.87658691, ..., 241.73791504,
        241.25744629, 240.79455566],
       [242.96936035, 242.95666504, 242.9420166 , ..., 241.98498535,
        241.4654541 , 240.96057129],
       [242.9888916 , 242.98205566, 242.97229004, ..., 242.20373535,
        241.65100098, 241.11193848],
       ...,
       [269.45666504, 269.48498535, 269.56213379, ..., 269.52893066,
        269.26428223, 268.98303223],
       [269.56994629, 269.55822754, 269.57873535, ..., 269.55822754,
        269.31115723, 269.04260254],
       [269.66662598, 269.63049316, 269.61682129, ..., 269.60900879,
        269.3560791 , 269.10314941]])

In [14]:
arr[4,0,7]

array([[242.90100098, 242.8873291 , 242.87658691, ..., 241.73791504,
        241.25744629, 240.79455566],
       [242.96936035, 242.95666504, 242.9420166 , ..., 241.98498535,
        241.4654541 , 240.96057129],
       [242.9888916 , 242.98205566, 242.97229004, ..., 242.20373535,
        241.65100098, 241.11193848],
       ...,
       [269.45666504, 269.48498535, 269.56213379, ..., 269.52893066,
        269.26428223, 268.98303223],
       [269.56994629, 269.55822754, 269.57873535, ..., 269.55822754,
        269.31115723, 269.04260254],
       [269.66662598, 269.63049316, 269.61682129, ..., 269.60900879,
        269.3560791 , 269.10314941]])

In [15]:
inTensor=arr.reshape(numDays*framesPerDay,*gridShape,numChannels*numIsoLvls)

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, TimeDistributed, LSTM

# Define your model
model = Sequential()

# Assuming you flatten the equipotential surface and attribute dimensions into the channel dimension
# Input shape: (Time, Height, Width, Channels) where Channels = 5 equipotential surfaces * 12 attributes
# You may need to adjust the input shape based on how you preprocess your data

# TimeDistributed Wrapper allows the same CNN to be applied to each time step independently
model.add(TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'), input_shape=(None, *gridShape, 60)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model.add(TimeDistributed(Conv2D(filters=64, kernel_size=(3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model.add(TimeDistributed(Flatten()))

# Add LSTM layer to capture temporal dependencies - consider this for later stages
# model.add(LSTM(units=50))

model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2))  # Predicting 2 values for the cyclone's next location; adjust based on your output

model.compile(optimizer='adam', loss='mean_squared_error')  # Use mean squared error for regression problems

model.summary()

# Training the model will require your data to be in the shape expected by the input layer
# X_train: Your feature data, y_train: Your target data (cyclone's next location or movement category)
# model.fit(X_train, y_train, epochs=10, batch_size=32)

c:\Users\KelM\anaconda3\envs\ecmwf\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed                │ (None, None, 219, 379, │        17,312 │
│ (TimeDistributed)               │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, None, 109, 189, │             0 │
│ (TimeDistributed)               │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, None, 107, 187, │        18,496 │
│ (TimeDistributed)               │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, None, 53, 93,   │             0 │
│ (TimeDistributed)               │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, None, 315456)   │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 100)      │    31,545,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, None, 2)        │           202 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,581,710 (120.47 MB)

 Trainable params: 31,581,710 (120.47 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
import pandas as pd

df = pd.read_csv('ibtracs.EP.list.v04r00.csv')

C:\Users\KelM\AppData\Local\Temp\ipykernel_3748\2479941462.py:3: DtypeWarning: Columns (1,2,8,9,14,19,20,161,162) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ibtracs.EP.list.v04r00.csv')


In [36]:
df.columns[:20]

Index(['SID', 'SEASON', 'NUMBER', 'BASIN', 'SUBBASIN', 'NAME', 'ISO_TIME',
       'NATURE', 'LAT', 'LON', 'WMO_WIND', 'WMO_PRES', 'WMO_AGENCY',
       'TRACK_TYPE', 'DIST2LAND', 'LANDFALL', 'IFLAG', 'USA_AGENCY',
       'USA_ATCF_ID', 'USA_LAT'],
      dtype='object')

In [33]:
df['NATURE'][0:99]

0       
1     TS
2     TS
3     TS
4     TS
      ..
94    TS
95    TS
96    TS
97    TS
98    TS
Name: NATURE, Length: 99, dtype: object

In [41]:
df_trim=df[['SID','NAME','SEASON','NUMBER','NATURE','ISO_TIME','LAT','LON','WMO_WIND','WMO_PRES','USA_RECORD','USA_STATUS','USA_WIND','USA_PRES','USA_GUST']][1:]
df_trim['SEASON']=pd.to_numeric(df_trim['SEASON'])
df_trim=df_trim[df_trim['SEASON']>=2015]

In [42]:
df_trim.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13812 entries, 82432 to 96243
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   SID         13812 non-null  object
 1   NAME        13812 non-null  object
 2   SEASON      13812 non-null  int64 
 3   NUMBER      13812 non-null  object
 4   NATURE      13812 non-null  object
 5   ISO_TIME    13812 non-null  object
 6   LAT         13812 non-null  object
 7   LON         13812 non-null  object
 8   WMO_WIND    13812 non-null  object
 9   WMO_PRES    13812 non-null  object
 10  USA_RECORD  13812 non-null  object
 11  USA_STATUS  13812 non-null  object
 12  USA_WIND    13812 non-null  object
 13  USA_PRES    13812 non-null  object
 14  USA_GUST    13812 non-null  object
dtypes: int64(1), object(14)
memory usage: 1.7+ MB


In [43]:
df_trim.to_csv('BestTrack_Trimmed.csv')

In [17]:
#import cdsapi
#c = cdsapi.Client()
#c.retrieve()